In [1]:
import pandas as pd
import json
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

df_MCA = pd.read_excel("/Users/azaruddin.sk/Desktop/YUbi/NLP Task/MCA.xlsx")
df_Beureu = pd.read_excel("/Users/azaruddin.sk/Desktop/YUbi/NLP Task/Bureau (1).xlsx")

df_MCA

,company_master_id,companyName,company_address
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I..."
1,569,PRAG BOSIMI SYNTHETICS LTD,"House no. 19, Ambikagiri Nagar, Milan Path, ..."
2,580,NATIONAL BOARDS LIMITED,PANIKHAITI GUWAHATI Assam 781026 India
3,581,NATIONAL PLYWOOD INDUSTRIES LTD,MAKUM PATHAR MARGHERITA ASSAM Tinsukia Assam...
4,582,KITPLY INDUSTRIES LIMITED,MAKUM PATHAR A.T.ROAD MARGHERITA Assam 78618...
...,...,...,...
12698,1088111,SHRIYA SALES LLP,"ROOM NO. 3, BANGALI CHAWL, KOLDONGARI, SAHAR R..."
12699,1093979,TASTY TRADERS LLP,"House No 354, Ariyakkattu House Thalore Post..."
12700,1097670,VIPUL TRADERS LLP,"438, Near Water Tank samta colony RAIPUR Rai..."
12701,1099949,DIMPLES CINE ADVERTISING PRIVATE LIMITED,PARK AVENUE16/29TH RD JUNCTION NR PALI NAKA...


In [2]:
df_Beureu = df_Beureu.groupby('company_master_id').agg({'company_address_1':lambda x:list(set(x))}).reset_index()

df_Beureu
#groupby = list of adress, set removes duplicates from dataframe

,company_master_id,company_address_1
0,522,"[LOSS, 52, Chowringhee Road , Kolkata-71, nan,..."
1,569,"[RANGPUR, RANGPUR PATH, R G BARUAH, BARUAH ROA..."
2,580,"[PANIKHAITI GUWAHATI 781026, PANIKHATI, GUWAH..."
3,581,"[MAKUM ROAD, TINSUKIA, MAKUM ROAD TINSUKIA A..."
4,582,"[RUNGAGORA ROAD, TINSUKIA-786 125, ASSAM, Rung..."
...,...,...
12698,1088111,"[NH 6, NEW GOVT MILK SCHEME, MURTIJAPUR ROAD, ..."
12699,1093979,"[Kunnukuizhy_x000D_\nTC 12/18/1, Therrinmoodu,..."
12700,1097670,"[Village Jaspal Bangar,Kanganwal Road,Ludhiana..."
12701,1099949,"[Park Avenue, 16th/29th Road Junction, Nr Pali..."


In [3]:
df_merged = df_MCA.merge(df_Beureu[['company_master_id', 'company_address_1']], 
                    on='company_master_id', how='left', suffixes=('', '_1'))

In [4]:
df_merged = df_merged.explode('company_address_1')

#explode = multiplies the list of entries creates multiple example like 522
df_merged

,company_master_id,companyName,company_address,company_address_1
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...",LOSS
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, Chowringhee Road , Kolkata-71"
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...",NaN
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, CHOWRINGHEE ROAD, KOLKATTA-700071"
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...",̵
...,...,...,...,...
12700,1097670,VIPUL TRADERS LLP,"438, Near Water Tank samta colony RAIPUR Rai...","Village Jaspal Bangar,Kanganwal Road,Ludhiana,..."
12701,1099949,DIMPLES CINE ADVERTISING PRIVATE LIMITED,PARK AVENUE16/29TH RD JUNCTION NR PALI NAKA...,"Park Avenue, 16th/29th Road Junction, Nr Pali ..."
12701,1099949,DIMPLES CINE ADVERTISING PRIVATE LIMITED,PARK AVENUE16/29TH RD JUNCTION NR PALI NAKA...,"PARK AVENUE, 16/29TH RD JUNCTION NR PALI, NAKA..."
12702,2127674,JATALIA GLOBAL VENTURES LIMITED,"500, 5th Floor, ITL Twin Tower, Netaji Subha...","DEEPALPUR ROAD, BAHALGARH, DISTT. SONEPAT"


In [5]:
df_merged = df_merged.dropna(subset = 'company_address_1').reset_index(drop = True)

In [6]:
df_merged

,company_master_id,companyName,company_address,company_address_1
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...",LOSS
1,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, Chowringhee Road , Kolkata-71"
2,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, CHOWRINGHEE ROAD, KOLKATTA-700071"
3,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...",̵
4,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, CHOWRINGHEE ROAD , KOLKATA-71"
...,...,...,...,...
37163,1097670,VIPUL TRADERS LLP,"438, Near Water Tank samta colony RAIPUR Rai...","Village Jaspal Bangar,Kanganwal Road,Ludhiana,..."
37164,1099949,DIMPLES CINE ADVERTISING PRIVATE LIMITED,PARK AVENUE16/29TH RD JUNCTION NR PALI NAKA...,"Park Avenue, 16th/29th Road Junction, Nr Pali ..."
37165,1099949,DIMPLES CINE ADVERTISING PRIVATE LIMITED,PARK AVENUE16/29TH RD JUNCTION NR PALI NAKA...,"PARK AVENUE, 16/29TH RD JUNCTION NR PALI, NAKA..."
37166,2127674,JATALIA GLOBAL VENTURES LIMITED,"500, 5th Floor, ITL Twin Tower, Netaji Subha...","DEEPALPUR ROAD, BAHALGARH, DISTT. SONEPAT"


In [7]:
def get_address_match_score(company_address, company_address_1, session):
    url = "https://ml-qa.go-yubi.in/address-match/invocations"
    
    payload = json.dumps({
        "address1": company_address,
        "address2": company_address_1
    })
    headers = {
        'Content-Type': 'application/json'
    }
    
    try:
        response = session.post(url, headers=headers, data=payload, timeout=300)
        if response.status_code == 200:
            response_data = response.json()
            return response_data.get("match_score")
        else:
            print(f"Error: Received status code {response.status_code} for addresses: {company_address}, {company_address_1}")
            return -1
        
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return -1
    
    
def parallel_apply(df, func, *args, **kwargs):
    results = []
#     print(df.columns)
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {
            executor.submit(func, row['company_address '], row['company_address_1'], *args, **kwargs): i
            for i, row in df.iterrows()
        }
        
        for future in tqdm(as_completed(futures), total=len(futures)):
            index = futures[future]
            try:
                results.append((index, future.result()))
            except Exception as e:
                    results.append((index, (e, None)))
    return results


with requests.Session() as session:
    address_match_results = parallel_apply(df_merged, get_address_match_score, session)

100%|█████████████████████████████████████| 37168/37168 [20:50<00:00, 29.72it/s]


In [13]:
for index, result in address_match_results:
    score = result
    df_merged.loc[index, 'match_score'] = score

In [16]:
df_merged

,company_master_id,companyName,company_address,company_address_1,match_score
0,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...",LOSS,0.011905
1,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, Chowringhee Road , Kolkata-71",0.000000
2,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, CHOWRINGHEE ROAD, KOLKATTA-700071",0.000000
3,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...",̵,0.000000
4,522,ASSAM COMPANY INDIA LIMITED,"GREENWOOD TEA ESTATE, DIBRUGARH Assam 786001 I...","52, CHOWRINGHEE ROAD , KOLKATA-71",0.000000
...,...,...,...,...,...
37163,1097670,VIPUL TRADERS LLP,"438, Near Water Tank samta colony RAIPUR Rai...","Village Jaspal Bangar,Kanganwal Road,Ludhiana,...",0.032407
37164,1099949,DIMPLES CINE ADVERTISING PRIVATE LIMITED,PARK AVENUE16/29TH RD JUNCTION NR PALI NAKA...,"Park Avenue, 16th/29th Road Junction, Nr Pali ...",0.545455
37165,1099949,DIMPLES CINE ADVERTISING PRIVATE LIMITED,PARK AVENUE16/29TH RD JUNCTION NR PALI NAKA...,"PARK AVENUE, 16/29TH RD JUNCTION NR PALI, NAKA...",0.805556
37166,2127674,JATALIA GLOBAL VENTURES LIMITED,"500, 5th Floor, ITL Twin Tower, Netaji Subha...","DEEPALPUR ROAD, BAHALGARH, DISTT. SONEPAT",0.038235


In [17]:
df_merged.to_excel('/Users/azaruddin.sk/Desktop/YUbi/NLP Task/Adress_Matching.xlsx', index = False)